### Using `numba.jit` to speedup the computation of the Euclidean distance matrix 

In this notebook we implement a function to compute the Euclidean distance matrix using Numba's *just-in-time* compilation decorator. We compare it with the NumPy function we wrote before.

We will use two Numba functions here: The decorator ` @numba.jit` and `numba.prange`.

<a href="https://colab.research.google.com/github/Ziaeemehr/workshop_hpcpy/blob/main/notebooks/numba/euclidean-distance-matrix-numba.jit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


$$
d(i, j) = \sqrt{\sum_{k=1}^{m} \left( x_{ik} - y_{jk} \right)^2}
$$

In [4]:
import numpy as np
import numba

numba.set_num_threads(1)

In [5]:
@numba.jit(nopython=True, parallel=True)
def euclidean_numba1(x, y):
    """Implementation with numba."""

    nrows, ncols = x.shape
    dist_matrix = np.zeros((nrows, nrows))
    for i in range(nrows):
        for j in range(nrows):
            r = 0.0
            for k in numba.prange(ncols):
                r += (x[i][k] - y[j][k])**2
            dist_matrix[i][j] = r

    return dist_matrix


@numba.jit(nopython=True, parallel=True)
def euclidean_numba2(x, y):
    """Implementation with numba and numpy."""

    nrows, ncols = x.shape
    dist_matrix = np.zeros((nrows, nrows))
    for i in range(nrows):
        for j in numba.prange(nrows):
            dist_matrix[i][j] = ((x[i] - y[j])**2).sum()

    return dist_matrix

Let's include here our numpy implementation for comparison.

In [6]:
def euclidean_numpy(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = (x_ij - y_ij)^2
    """

    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[:, np.newaxis].T

    xy = np.dot(x, y.T)

    return np.abs(x2 + y2 - 2. * xy)

### Note
Observe that the inner loop, which is a reduction, is done with `numba.prange`. `numba.prange` automatically takes care of data privatization and reductions.

### Exercise 1
Before runing the different functions, could you say which of the two numba implementations would be faster?

In [7]:
# Let's check that they all give the same result
rng = np.random.default_rng()
x = 10. * rng.random((100, 10))

print(np.abs(euclidean_numpy(x, x) - euclidean_numba1(x, x)).max())
print(np.abs(euclidean_numpy(x, x) - euclidean_numba2(x, x)).max())

2.8421709430404007e-13
2.8421709430404007e-13


Our Numba implementations can be faster than the NumPy one for a list of small vectors. However, with larger vectors, the NumPy implementation is faster:

In [19]:
def timing(nrow, ncols):
    
    x = 10. * rng.random((nrow, ncols))
    print(f"{'numpy':<10}", end=" ")
    %timeit euclidean_numpy(x, x)
    print(f'{f"numba1":<10}', end=" ")
    %timeit euclidean_numba1(x, x)
    print(f"{f'numba2':<10}", end=" ")
    %timeit euclidean_numba2(x, x)

In [21]:
timing(100, 3)

numpy      72.1 μs ± 958 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
numba1     11 ms ± 298 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
numba2     154 μs ± 3.27 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In a more realistic case, our NumPy implementation is much faster:

In [22]:
timing(100, 10)

numpy      75.6 μs ± 3.85 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
numba1     11 ms ± 84.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
numba2     216 μs ± 807 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [25]:
nrows = 5000
ncols = 50

x = 10. * rng.random((nrows, ncols))

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)